In [1]:
# IMPORTANT 
#Install dependencies and copy model weights to run the notebook without internet access when submitting to the competition.

!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!pip install --no-index /kaggle/input/imc2025-deps/imc2025-deps/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/resnet60/pytorch/default/1/resnet50-0676ba61.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/superpoint/pytorch/default/1/superpoint_v1.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!mkdir -p /root/.cache/torch/hub/netvlad/netvlad
!cp /kaggle/input/netvlad/other/default/1/Pitts30K_struct.mat /root/.cache/torch/hub/netvlad/VGG16-NetVLAD-Pitts30K.mat
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv.pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.8
    Uninstalling kornia_rs-0.1.8:
      Successfully uninstalled kornia_rs-0.1.8
  Attempting uninstall: kornia
    Found existing installation: kornia 0.8.0
    Uninstalling kornia-0.8.0:
      Successfully uninstalled ko

In [2]:
!python -m pip install --no-index --find-links=/kaggle/input/pkg-check-orientation/ check_orientation==0.0.5 > /dev/null
!cp /kaggle/input/pkg-check-orientation/2020-11-16_resnext50_32x4d.zip /root/.cache/torch/hub/checkpoints/

In [3]:
!pip uninstall -y lightglue

Found existing installation: lightglue 0.0
Uninstalling lightglue-0.0:
  Successfully uninstalled lightglue-0.0


In [4]:
!cp -r /kaggle/input/hloc-with-rdd/pytorch/default/22/Hierarchical-Localization /root/
!cd /root/Hierarchical-Localization && python -m pip install -e .
import sys
sys.path.append('/root/Hierarchical-Localization')

Obtaining file:///root/Hierarchical-Localization
  Preparing metadata (setup.py) ... done
  Running setup.py develop for hloc


In [5]:
! cd /root/Hierarchical-Localization/third_party/rdd/RDD/models/ops && python -m pip install -e .
sys.path.append('/root/Hierarchical-Localization/third_party/rdd/RDD/models/ops')

Obtaining file:///root/Hierarchical-Localization/third_party/rdd/RDD/models/ops
  Preparing metadata (setup.py) ... done
  Running setup.py develop for MultiScaleDeformableAttention


In [6]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

# IMPORTANT Utilities: importing data into colmap and competition metric
import pycolmap
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric
from pathlib import Path
import itertools
from concurrent.futures import ThreadPoolExecutor
from torch.backends import cudnn
import importlib
import shutil
from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_retrieval,
    pairs_from_exhaustive
)
from hloc.reconstruction import *
from hloc.utils.parsers import names_to_pair, names_to_pair_old, parse_retrieval
from torch.utils.data import Subset

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [7]:
# Do not forget to select an accelerator on the sidebar to the right.
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

device=device(type='cuda', index=0)


In [8]:
# Collect vital info from the dataset

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

# Set is_train=True to run the notebook on the training data.
# Set is_train=False if submitting an entry to the competition (test data is hidden, and different from what you see on the "test" folder).
is_train = False
data_dir = '/kaggle/input/image-matching-challenge-2025'
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

Dataset "ETs" -> num_images=22
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "stairs" -> num_images=51


In [9]:
def remove_duplicate_pairs(pairs_path):
    pairs = parse_retrieval(pairs_path)
    pairs = [(q, r) for q, rs in pairs.items() for r in rs]
    pairs = match_features.find_unique_new_pairs(pairs, None)
    os.system(f'rm -rf {str(pairs_path)}')
    with open(pairs_path, 'w') as f:
        f.write("\n".join(" ".join([i, j]) for i, j in pairs))

In [10]:
def split_pairs(path):
    path = Path(path)
    with open(path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    mid = len(lines) // 2
    pairs_split1 = lines[:mid]
    pairs_split2 = lines[mid:]

    # Define output file paths
    output1 = path.parent / (path.stem + '_1.txt')
    output2 = path.parent / (path.stem + '_2.txt')

    # Write each half to its own file
    with open(output1, 'w') as f:
        f.write('\n'.join(pairs_split1) + '\n')
    with open(output2, 'w') as f:
        f.write('\n'.join(pairs_split2) + '\n')

    return output1, output2


def combine_pairs(path1, path2, output_path):
    path1 = Path(path1)
    path2 = Path(path2)
    output_path = Path(output_path)

    # Read lines from both input files
    with open(path1, 'r') as f1:
        lines1 = [line.strip() for line in f1 if line.strip()]

    with open(path2, 'r') as f2:
        lines2 = [line.strip() for line in f2 if line.strip()]

    # Combine the lines
    combined_lines = lines1 + lines2

    # Write to output file
    with open(output_path, 'w') as out_f:
        out_f.write('\n'.join(combined_lines) + '\n')

    return output_path

# Classification

In [11]:
config = {
    "d_model": 256,
    "nhead": 8,
    "layer_names": ["self", "cross"] * 4,
    "attention": "linear",
    "input_dim": 768,
}
sys.path.append('/kaggle/input/kaggle-classifier/kaggle-classifier')
from model.transformer import LocalFeatureTransformer

def get_descriptors(path: Path, name: str) -> np.ndarray:
    with h5py.File(str(path), "r", libver="latest") as hfile:
        dset = hfile[name]["descriptors"]
        return dset[:]
        
def exec_pair_classification(sfm_pairs, device, feature_path, output_path):
    model = LocalFeatureTransformer(config)
    model.load_state_dict(torch.load('/kaggle/input/kaggle-classifier/kaggle-classifier/checkpoints/checkpoint_epoch_16.pth')['model_state_dict'])
    model.eval().to(device)
    pairs = parse_retrieval(sfm_pairs)
    pairs = [(q, r) for q, rs in pairs.items() for r in rs]
    pairs = match_features.find_unique_new_pairs(pairs, None)
    new_pairs = []
    with torch.no_grad():
        for i, (q, r) in tqdm(enumerate(pairs)):
            desc_q = get_descriptors(feature_path, q)
            desc_r = get_descriptors(feature_path, r)
            desc_q = torch.from_numpy(desc_q).to(device)
            desc_r = torch.from_numpy(desc_r).to(device)
            desc_q = desc_q.unsqueeze(0).to(torch.float32)
            desc_r = desc_r.unsqueeze(0).to(torch.float32)
            out = model(desc_q, desc_r)
            if out[0][0] >= 0.6:
                new_pairs.append((q, r))
    
    with open(output_path, 'w') as f:
        f.write("\n".join(" ".join([i, j]) for i, j in new_pairs))
    return output_path

# Rotation

In [12]:
from torchvision.io import read_image as T_read_image
from torchvision.io import ImageReadMode
from torchvision import transforms as T
from check_orientation.pre_trained_models import create_model
from torch.utils.data import Dataset, DataLoader

def convert_rot_k(index):
    if index == 0:
        return 0
    elif index == 1:
        return 3
    elif index == 2:
        return 2
    else:
        return 1

class CheckRotationDataset(Dataset):
    def __init__(self, files, transform=None):
        self.transform = transform
        self.files = files

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        imgPath = self.files[idx]
        image = T_read_image(imgPath, mode=ImageReadMode.RGB)
        if self.transform:
            image = self.transform(image)
        return image

def get_CheckRotation_dataloader_crop(images, batch_size=1):
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ConvertImageDtype(torch.float),
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    dataset = CheckRotationDataset(images, transform=transform)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def exec_rotation_detection(img_files, device):
    model = create_model("swsl_resnext50_32x4d")
    model.eval().to(device);
    
    dataloader = get_CheckRotation_dataloader_crop(img_files)
    
    rots = []
    for idx, image in enumerate(dataloader):
        image = image.to(torch.float32).to(device)
        with torch.no_grad():
            prediction = model(image).detach().cpu().numpy()
            detected_rot = prediction[0].argmax()
            if prediction[0][detected_rot] > 0.9:
                rot_k = convert_rot_k(detected_rot)
            else:
                rot_k = 0
            rots.append(rot_k)
            print(f"{os.path.basename(img_files[idx])} > rot_k={rot_k}")
    return rots

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


In [13]:
def output_rot_images(
    paths,
    output_dir,
    rots,
):
    corrected_image_paths = []
    for rot, path in tqdm(zip(rots, paths), total=len(paths), desc=f"Rotating images", dynamic_ncols=True):
        try:
            img = cv2.imread(str(path))
    
            if rot == 1:
                img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            elif rot == 2:
                img = cv2.rotate(img, cv2.ROTATE_180)
            elif rot == 3:
                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    
            cv2.imwrite(str(output_dir / path.name), img)
        except:
            shutil.copy(str(path), str(output_dir / path.name))
        
        corrected_image_paths.append(output_dir / path.name)

    return corrected_image_paths

def exec_rotation_correction(paths, output_dir):

    rots = exec_rotation_detection(paths, device)

    corrected_image_paths = []
    with ThreadPoolExecutor() as executor:
        results = executor.map(
            output_rot_images,
            np.array_split(paths, 2),
            itertools.repeat(output_dir),
            np.array_split(rots, 2),
        )
        for data in results:
            corrected_image_paths.append(data)

    corrected_image_paths = list(itertools.chain.from_iterable(corrected_image_paths))
    gc.collect()

    return corrected_image_paths

# Run All

In [14]:
from hloc.utils.io import get_keypoints, get_matches, names_to_pair

def merge_keypoints(image_list, feature_path1, feature_path2, save_name='feats-merged.h5'):
    output = feature_path1.parent / save_name
    with h5py.File(str(output), "a", libver="latest") as fd:
        for name in image_list:
            try:
                if name in fd:
                    del fd[name]
                grp = fd.create_group(name)
                kpts1 = get_keypoints(feature_path1, name)
                if isinstance(feature_path2, list):
                    try:
                        kpts2 = get_keypoints(feature_path2[0], name)
                    except:
                        kpts2 = get_keypoints(feature_path2[1], name)
                else:
                    kpts2 = get_keypoints(feature_path2, name)
                kpts = np.concatenate((kpts1, kpts2), axis=0)
                len1 = kpts1.shape[0]
                grp.create_dataset('keypoints', data=kpts)
                grp.create_dataset('length', data=len1)
                
            except OSError as error:
                if "No space left on device" in error.args[0]:
                    logger.error(
                        "Out of disk space: storing features on disk can take "
                        "significant space, did you enable the as_half flag?"
                    )
                    del grp, fd[name]
                raise error
    return output

def get_length(feature_path, name):
    with h5py.File(str(feature_path), "r", libver="latest") as fd:
        length = fd[name]['length'][()]
    return length

def merge_matches(matches_path, matches_path_sg, pairs_path, merge_keypoints, save_name='matches-merged.h5'):
    output = matches_path.parent / save_name
    with open(str(pairs_path), "r") as f:
        pairs = [p.split() for p in f.readlines()]
    
    with h5py.File(str(output), "a", libver="latest") as fd:
        for name0, name1 in tqdm(pairs):
            pair = names_to_pair(name0, name1)
            
            try:
                out1 = get_matches(matches_path, name0, name1)
                    
                matches1 = out1[0]
                scores1 = out1[1]

                out2 = get_matches(matches_path_sg, name0, name1)
                matches2 = out2[0]
                scores2 = out2[1]

                length1 = get_length(merge_keypoints, name0)
                length2 = get_length(merge_keypoints, name1)
                
                # add length to matches2
                matches2[:, 0] += length1
                matches2[:, 1] += length2
                
                matches = np.concatenate((matches1, matches2), axis=0)
            
                scores = np.concatenate((scores1, scores2), axis=0)

                with h5py.File(str(output), "a", libver="latest") as fd:
                    if pair in fd:
                        del fd[pair]
                    grp = fd.create_group(pair)
                    grp.create_dataset("matches0", data=matches)
                    grp.create_dataset("matching_scores0", data=scores)
            except OSError as error:
                if "No space left on device" in error.args[0]:
                    logger.error(
                        "Out of disk space: storing features on disk can take "
                        "significant space, did you enable the as_half flag?"
                    )
                    del grp, fd[pair]
                raise error
    return output

In [15]:
gc.collect()

max_images = None  # Used For debugging only. Set to None to disable.
datasets_to_process = None  # Not the best convention, but None means all datasets.

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	#'amy_gardens',
    	'ETs',
    	#'fbk_vineyard',
    	'stairs',
    	#Data from IMC 2023 and 2024.
    	#'imc2024_dioscuri_baalshamin',
    	#'imc2023_theather_imc2024_church',
    	#'imc2023_heritage',
    	#'imc2023_haiper',
    	#'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	#'pt_stpeters_stpauls',
    	#'pt_brandenburg_british_buckingham',
    	#'pt_piazzasanmarco_grandplace',
    	#'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []


print (f"Extracting on device {device}")
for dataset, predictions in samples.items():
    if datasets_to_process and dataset not in datasets_to_process:
        print(f'Skipping "{dataset}"')
        continue

    images_dir = Path(os.path.join(data_dir, 'train' if is_train else 'test', dataset))
    image_paths = list(images_dir.glob('**/*.[pjJP][npNP][gG]'))  # Matches .jpg, .png, .JPG, .PNG, etc.

    if not images_dir.exists():
        continue
    
    images = os.listdir(images_dir)
    images = [image for image in images if image.endswith('.jpg') or image.endswith('.png')]
    images_path = [images_dir / image for image in images]

    mid = len(images) // 2
    images_split1 = images[:mid]
    images_split2 = images[mid:]
    
    feature_dir = os.path.join(workdir, 'featureout', dataset)
    os.makedirs(feature_dir, exist_ok=True)

    outputs = Path(feature_dir)
        
    # check for rotation
    
    corrected_images_dir = outputs / "corrected_images"
    corrected_images_dir.mkdir(parents=True, exist_ok=True)
    exec_rotation_correction(image_paths, corrected_images_dir)
    corrected_images_paths = list(corrected_images_dir.glob('**/*.[pjJP][npNP][gG]'))
    torch.cuda.empty_cache()
    gc.collect()
    
    sfm_pairs = outputs / "pairs-all.txt"
    sfm_pairs_retrival = outputs / "pairs-netvlad.txt"
    sfm_dir = outputs / "sfm_rdd+lightglue"
    
    ##### rdd+lightglue
    
    feature_conf1 = {'model': {'combine': False, 'max_keypoints': 8192, 'name': 'rdd'},
     'output': 'feats-rdd-r1024',
     'preprocessing': {'grayscale': False, 'resize_max': 1024, 'resize_force': True}}
    feature_conf2 = {'model': {'combine': False, 'max_keypoints': 8192, 'name': 'rdd'},
     'output': 'feats-rdd-r1280',
     'preprocessing': {'grayscale': False, 'resize_max': 1280, 'resize_force': True}}
    
    matcher_conf = {'model': {'depth_confidence': -1,
                               'features': 'rdd',
                               'mp': True,
                               'name': 'lightglue',
                               'width_confidence': 0.99},
                     'output': 'matches-rdd-lightglue'}

    matcher_conf2 = {'model': {'depth_confidence': -1,
                               'features': 'rdd',
                               'mp': True,
                               'name': 'lightglue',
                               'width_confidence': 0.99},
                     'output': 'matches-rdd-lightglue'}
    
    print(f'\nProcessing dataset "{dataset}": {len(os.listdir(corrected_images_dir))} images')
    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}
    
    pairs_from_exhaustive.main(sfm_pairs, images)
    retrieval_conf = extract_features.confs["netvlad"]
    retrieval_path = extract_features.main(retrieval_conf, corrected_images_dir, outputs, device='0')
    num_matched = max(20, int(0.12 * len(images)))
    pairs_from_retrieval.main(retrieval_path, sfm_pairs_retrival, num_matched=num_matched)

    dino_conf = {
        "output": "global-feats-dino",
        "model": {"name": "dino", "model_path": "/kaggle/input/dinov2/pytorch/base/1"},
        "preprocessing": {"resize_max": 1024, "grayscale": False},
    }
    dino_path = extract_features.main(dino_conf, corrected_images_dir, outputs, device='0', overwrite=True)
    
    # remove_duplicate_pairs(sfm_pairs)
    sfm_pairs1, sfm_pairs2 = split_pairs(sfm_pairs)

    sfm_pairs1_new = sfm_pairs1.parent / (sfm_pairs1.stem + '_new.txt')
    sfm_pairs2_new = sfm_pairs2.parent / (sfm_pairs2.stem + '_new.txt')
    

    with ThreadPoolExecutor() as executor:
        sfm_pairs_list = list(executor.map(
            exec_pair_classification,
            [sfm_pairs1, sfm_pairs2],
            ['cuda:0', 'cuda:1'],
            [dino_path, dino_path],
            [sfm_pairs1_new, sfm_pairs2_new]
        ))


    sfm_pairs_new = sfm_pairs.parent / (sfm_pairs.stem + '_new.txt')
    combine_pairs(sfm_pairs1_new, sfm_pairs2_new, sfm_pairs_new)

    sfm_pairs_combine = sfm_pairs.parent / (sfm_pairs.stem + '_final.txt')
    combine_pairs(sfm_pairs_new, sfm_pairs_retrival, sfm_pairs_combine)

    remove_duplicate_pairs(sfm_pairs_combine)

    print('feature_conf1', feature_conf1)
    print('feature_conf2', feature_conf2)
    feature_path1 = extract_features.main(feature_conf1, corrected_images_dir, outputs, device='0', overwrite=True)
    feature_path2 = extract_features.main(feature_conf2, corrected_images_dir, outputs, device='0', overwrite=True)
    
    # remove_duplicate_pairs(sfm_pairs)
    
    # sfm_pairs1, sfm_pairs2 = split_pairs(sfm_pairs)
    
    with ThreadPoolExecutor() as executor:
        match_paths = list(executor.map(
            match_features.main,
            [matcher_conf, matcher_conf2],
            [sfm_pairs_combine, sfm_pairs_combine],
            [feature_conf1["output"], feature_conf2["output"]],
            [outputs, outputs],
            [None, None],
            [None, None],
            [True, True],
            ['0', '1'],
            [0, 0]
        ))

    """sfm_pairs_new = sfm_pairs.parent / (sfm_pairs.stem + '_new.txt')
    combine_pairs(sfm_pairs1_new, sfm_pairs2_new, sfm_pairs_new)"""

    merged_features_path = merge_keypoints(images, feature_path1, feature_path2)
    merged_match_path = merge_matches(match_paths[0], match_paths[1], sfm_pairs_combine, merged_features_path, save_name='matches-merged.h5')
    
    sfm_dir.mkdir(parents=True, exist_ok=True)
    database = sfm_dir / "database.db"
    models_path = sfm_dir / "models"
    camera_mode = pycolmap.CameraMode.AUTO
    models_path.mkdir(exist_ok=True, parents=True)
    
    create_empty_db(database)
    import_images(corrected_images_dir, database, camera_mode, None, None)
    image_ids = get_image_ids(database)
    import_features(image_ids, database, merged_features_path)
    
    import_matches(
        image_ids,
        database,
        sfm_pairs_combine,
        merged_match_path,
        0.2,
        False,
    )
    
    estimation_and_geometric_verification(database, sfm_pairs_combine, False)
    
    os.makedirs(models_path, exist_ok=True)
    t = time()
    
    # Run the mapping
    
    new_database_path = database
    
    mapper_options = pycolmap.IncrementalPipelineOptions()
    mapper_options.min_model_size = 3
    mapper_options.max_num_models = 5
    
    maps = pycolmap.incremental_mapping(
        database_path=new_database_path, 
        image_path=corrected_images_dir,
        output_path=models_path,
        options=mapper_options)
    
    clear_output(wait=False)

    registered = 0
    for map_index, cur_map in maps.items():
        for index, image in cur_map.images.items():
            prediction_index = filename_to_index[image.name]
            predictions[prediction_index].cluster_index = map_index
            predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
            predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
            registered += 1
    mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(os.listdir(images_dir))} images with {len(maps)} clusters'
    mapping_result_strs.append(mapping_result_str)
    print(mapping_result_str)
    gc.collect()
    os.system(f'rm -rf {str(corrected_images_dir)}')

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

Dataset "stairs" -> Registered 18 / 52 images with 3 clusters

Results
Dataset "ETs" -> Registered 21 / 23 images with 2 clusters
Dataset "stairs" -> Registered 18 / 52 images with 3 clusters

Timings
shortlisting -> total=0.00 sec.
feature_detection -> total=0.00 sec.
feature_matching -> total=0.00 sec.
RANSAC -> total=0.00 sec.
Reconstruction -> total=0.00 sec.


In [16]:
# Must Create a submission file.
array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv' 
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')

!head {submission_file}

image_id,dataset,scene,image,rotation_matrix,translation_vector
ETs_another_et_another_et001.png_public,ETs,cluster1,another_et_another_et001.png,0.999913408;-0.004191294;0.012474385;0.003881054;0.999685114;0.024791343;-0.012574365;-0.024740782;0.999614815,-2.456640949;-1.763082609;3.953740239
ETs_another_et_another_et002.png_public,ETs,cluster1,another_et_another_et002.png,0.999994489;-0.001933476;-0.002698833;0.001931391;0.999997834;-0.000775126;0.002700326;0.000769909;0.999996058,-2.290432793;-0.884242681;2.170288956
ETs_another_et_another_et003.png_public,ETs,cluster1,another_et_another_et003.png,0.997992902;-0.041341131;0.047969550;0.044316919;0.997046357;-0.062726176;-0.045234694;0.064726142;0.996877299,-2.512209274;0.847355887;0.083296035
ETs_another_et_another_et004.png_public,ETs,cluster1,another_et_another_et004.png,0.999357827;-0.011543198;0.033921806;0.006984932;0.991285909;0.131542606;-0.035144631;-0.131221192;0.990729960,-2.399628676;-1.094903251;0.526141714
ETs_another_e

In [17]:
# Definitely Compute results if running on the training set.
# Do not do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')